---

(c) Author: Arthur Grigorjan, arthur.grigrojan@ipa.fraunhofer.de

---

# Solving Job Shop Scheduling Problems using QAOA on the IBM Q System One

The third SEQUOIA-Use-Case "Optimization of order sequencing in discrete manufacturing" deals with the distribution of production orders on different production resources.

This demonstrator shows an approach to solve the job shop scheduling problem (JSP) problem on the IBM Q System One.
The JSP deals with the distribution of production orders (jobs) on different production resources (machines).
This problem exists in almost all manufacturing industries, however, it usually varies in detail in the influencing factors to be mapped (e.g. material availability, employee skills, plant capacities, etc.), constraints or objectives to be optimized.
A widely used definition of this problem type in the literature is the job shop schedluing problem. This is a condensed version of the problem which defines the following influencing factors for a classical JSP:

There are a number of production orders (jobs) to be processed, where each job can consist of a number of production steps (operations) that must be processed in a specified order. Each production step must be performed on a specific piece of equipment (machine). The equipment is subject to a number of restrictions. For example, the previous manufacturing steps of each job must be completed before the current manufacturing step can be processed.
Also, a machine can only process one operation at any given time.
Operations cannot be paused. The goal of the classic JSP is to minimize the total production time (Makespan).


![Time indexed QUBO for the JSP](./img/time-indexed_qubo-JSP.PNG)

We use the time indexed QUBO to formulate the objective function of the job shop scheduling problem as described by Zhang et al. [1].

Desciption of the penalty term:

p4 - The penalty term p4 motivates the QUBO to start an operation as often as possible. This has to be mitigated by the other penalty erms.

p5 - The penalty term p5 motivates the QUBO to make sure that the processing order of all operations of a job is correct. (precedence contrainat) 

p6 - The penalty term p6 motivates the QUBO to make sure that at any given point of time there are not multiple operations beeing processed at a given machine. (resource contraint) 

p7 - The penalty term p7 motivates the QUBO to complete the last opeartion of each job as early as possible.



-----

[1] Zhang, J., Lo Bianco, G. and Beck, J. C. (2022) “Solving Job-Shop Scheduling Problems with QUBO-Based Specialized Hardware”, Proceedings of the International Conference on Automated Planning and Scheduling, 32(1), pp. 404-412. doi: 10.1609/icaps.v32i1.19826. 

https://ojs.aaai.org/index.php/ICAPS/article/view/19826


-----

# Import dependecies

In [1]:
import numpy as np
from typing import Dict, List, Optional, Tuple, Union, cast

# Import helper functions for the JSP (drawing, JSP data class, calculating makespan of a schedule etc.)
from jsp_utils import *

# Configure the parameters for a JSP instance

In [2]:
jsp_filepath = './data/agrg_minimal.txt'

##### The following parameters should be initialized by a heuristic in the future


In [3]:
# penalty term values

p4 = 1 # everyoperation has to start
p5 = 1 # precedence contrainats
p6 = 1 # resource contraint
p7 = 1 # no job exceed makespan


In [4]:
# The horizon has to be estimated in advance in a smart way...
horizon = 2 # T as described in the QUBO

# Upper and lower boundaries of the integer variables have to be estimated

# makespan int var for the cost function and penalty term 7
c_max_lb = int(horizon*0.0)
c_max_ub = horizon

# slack variables for penalty term 7
j_slack_lb = 0
j_slack_ub = 1


# Load JSP instance from file

In [5]:
# job_dict = readInstance('./data/ft03.txt')
job_dict = read_instance(jsp_filepath)

job_dict

[[(1, 1), (0, 1)], [(0, 1), (1, 1)]]

# Create our JSP object

In [6]:
jsp = JSP(job_dict)
jsp

(jId:0, ops:([(m:1, d:1, oId:0), (m:0, d:1, oId:1)])
(jId:1, ops:([(m:0, d:1, oId:0), (m:1, d:1, oId:1)])

# Define the quadratic programm which describes the JSP

#### Create all variables

In [7]:
from docplex.mp.model import Model

all_var_names = []

binary_var_names = []

mdl = Model('jssp time indexed qubo model')
i_var_c_max = mdl.integer_var(lb=c_max_lb, ub=c_max_ub, name='i_var_c_max')
all_var_names.append(i_var_c_max)

# We remove all absurd times for an operation. 
# The earliest start time of an operation is definetly 
# not earlier than the duration of the earlier operation of a specific job.
# 
# The latest start time of an operation is definetly not later than the predictied
# makespan Cmax minus the completion time of all later operations of a specific job.

length_of_operations_before = 0 # we keep track of the duration sum of the earlier operations of a job. 
for j in range(len(jsp.jobs)):
    for o in range(len(jsp.jobs[j].operations)):
        # calculate the sum of the duration of all operations 
        # in this job which have to be processed after this operation. 
        latest_start = horizon
        try:
            later_operations = jsp.jobs[j].operations[o+1:]
            duration_later_operations = sum( [op.duration for op in later_operations] )

            latest_start = horizon - duration_later_operations
        except IndexError:
            _ = 0
        

        # we only create start times which are not absurd here..
        for t in range(length_of_operations_before, latest_start): 
            v = mdl.binary_var(name=f"b_var_j{j}o{o}t{t}")
            all_var_names.append(v)
            binary_var_names.append(v)
            
        duration = jsp.jobs[j].operations[o].duration # increse the counter
        length_of_operations_before += duration
    
    length_of_operations_before = 0 # reset to 0 for the next job.


for j in range(len(jsp.jobs)):
    v = mdl.integer_var(lb=j_slack_lb, ub=j_slack_ub, name=f"i_var_slack_j{j}")
    all_var_names.append(v)


In [8]:
all_var_names

[docplex.mp.Var(type=I,name='i_var_c_max',ub=2),
 docplex.mp.Var(type=B,name='b_var_j0o0t0'),
 docplex.mp.Var(type=B,name='b_var_j0o1t1'),
 docplex.mp.Var(type=B,name='b_var_j1o0t0'),
 docplex.mp.Var(type=B,name='b_var_j1o1t1'),
 docplex.mp.Var(type=I,name='i_var_slack_j0',ub=1),
 docplex.mp.Var(type=I,name='i_var_slack_j1',ub=1)]

-----
#### Define penalty terms as described in the time indexed qubo

In [9]:
term_4 = []

# p4 term - every operation only starts once!
for j in range(len(jsp.jobs)):
    for o in range(len(jsp.jobs[j].operations)):
        vars_of_current_operation = [(x - 1) for x in all_var_names if f"b_var_j{j}o{o}" in x.name]
        s = mdl.sum(vars_of_current_operation)**2
        term_4.append(s)

term_4 = mdl.sum(term_4)

term_4

docplex.mp.quad.QuadExpr(b_var_j0o0t0^2+b_var_j0o1t1^2+b_var_j1o0t0^2+b_var_j1o1t1^2-2b_var_j0o0t0-2b_var_j0o1t1-2b_var_j1o0t0-2b_var_j1o1t1+4)

In [10]:
term_5 = []

#p5 term - precedence constraint - operations have to follow the right order

for j in range(len(jsp.jobs)):
    for o in range(len(jsp.jobs[j].operations)):

        # Stop before the last operation of this Job because the last operation has no next operation...
        if o+1 < len(jsp.jobs[j].operations) :

            o_this = jsp.jobs[j].operations[o] # current operation we look at
            o_next = jsp.jobs[j].operations[o+1] # next operation if any..
            d = o_this.duration

            for t1 in range(horizon):
                for t2 in range(horizon):
                    if (t1 + d) > t2:
                        try:
                            var1 = [x for x in all_var_names if f"b_var_j{j}o{o_this.id}t{t1}" in x.name][0]
                            # print(var1)
                            var2 = [x for x in all_var_names if f"b_var_j{j}o{o_next.id}t{t2}" in x.name][0]
                            # print(var2)

                            term_5.append(var1 * var2)
                        except IndexError:
                            # a variable could not be found because we removed it (absurd start times..)
                            # this is no problem..
                            _ = 1
                        

term_5 = mdl.sum(term_5)

term_5


docplex.mp.ZeroExpr()

In [11]:

term_6 = []

#p6 term - resource constraint - only one operation at a machine at a time..

# gather all operations which run on the same machine..
ops_of_machines = []

for m in range(jsp.get_number_of_machines()):
    ops_of_machines.append([])

for j in range(len(jsp.jobs)):
    for o in range(len(jsp.jobs[j].operations)):
        op = jsp.jobs[j].operations[o]
        ops_of_machines[op.machine].append(op)

# combination of all operations which run on the same machine...
combis = []
for m in range(len(ops_of_machines)):
    for o1 in ops_of_machines[m]:
        for o2 in ops_of_machines[m]:
            # check that o1 and o2 are not the same!
            if o1 is not o2:
                # don't append if combination is already appended in the other order..
                if (o2, o1) not in combis:
                    combis.append((o1, o2))


for c in combis:
    o1 = c[0]
    o2 = c[1]
    for t1 in range(horizon):
        for t2 in range(horizon):
            if t1 <= t2 and t2 < t1 + o1.duration:

                try:    
                    var1 = [x for x in all_var_names if f"b_var_j{o1.parent_job_id}o{o1.id}t{t1}" in x.name][0]
                    var2 = [x for x in all_var_names if f"b_var_j{o2.parent_job_id}o{o2.id}t{t2}" in x.name][0]
                    term_6.append(var1 * var2)
                except IndexError:
                    # a variable could not be found because we removed it (absurd start times..)
                    # this is ok
                    _ = 1   


term_6 = mdl.sum(term_6)

term_6

docplex.mp.ZeroExpr()

In [12]:
all_var_names

[docplex.mp.Var(type=I,name='i_var_c_max',ub=2),
 docplex.mp.Var(type=B,name='b_var_j0o0t0'),
 docplex.mp.Var(type=B,name='b_var_j0o1t1'),
 docplex.mp.Var(type=B,name='b_var_j1o0t0'),
 docplex.mp.Var(type=B,name='b_var_j1o1t1'),
 docplex.mp.Var(type=I,name='i_var_slack_j0',ub=1),
 docplex.mp.Var(type=I,name='i_var_slack_j1',ub=1)]

In [13]:
combis

[((m:0, d:1, oId:1), (m:0, d:1, oId:0)),
 ((m:1, d:1, oId:0), (m:1, d:1, oId:1))]

In [14]:
ops_of_machines

[[(m:0, d:1, oId:1), (m:0, d:1, oId:0)],
 [(m:1, d:1, oId:0), (m:1, d:1, oId:1)]]

In [15]:
term_7 = []

#p7 term - no job exceeds makespan!

for j in range(len(jsp.jobs)):
    ts = []
    o_last =  len(jsp.jobs[j].operations) - 1 
    d_last_op = jsp.jobs[j].operations[o_last].duration
    for t in range(horizon):
        
        try:    
            x_last_op_t = [x for x in all_var_names if f"b_var_j{j}o{o_last}t{t}" in x.name][0]
            ts.append( t * x_last_op_t )
        except IndexError:
            # a variable could not be found because we removed it (absurd start times..)
            # this is ok..
            _ = 1   
            
    slack_var_for_this_job = [x for x in all_var_names if f"i_var_slack_j{j}" in x.name][0]
    term_7.append( (mdl.sum(ts) + d_last_op + slack_var_for_this_job - i_var_c_max)**2 )

term_7 = mdl.sum(term_7)

term_7


docplex.mp.quad.QuadExpr(2i_var_c_max^2-2i_var_c_max*b_var_j0o1t1-2i_var_c_max*b_var_j1o1t1-2i_var_c_max*i_var_slack_j0-2i_var_c_max*i_var_slack_j1+b_var_j0o1t1^2+2b_var_j0o1t1*i_var_slack_j0+b_var_j1o1t1^2+2b_var_j1o1t1*i_var_slack_j1+i_var_slack_j0^2+i_var_slack_j1^2+-4i_var_c_max+2b_var_j0o1t1+2b_var_j1o1t1+2i_var_slack_j0+2i_var_slack_j1+2)

### Define the objective function 

#### As described in the time indexed QUBO

In [16]:
# Objective function as described in the time indexed qubo
mdl.minimize(i_var_c_max + p4 * term_4 + p5 * term_5 + p6 * term_6 + p7 * term_7)

#### Alternative formulation - replacing p4 with linear equality constraints

In [17]:
# '''
# Alternative QUBO formulation  
# We replace the p4-term with linear equality constraints to see if this performs better.
# Every operation has to start exactly once in this formulation. 
# Constraint will be converted later into a penalty term using the LinearEqualityToPenalty-Converter.
# '''
# for j in range(len(jsp.jobs)):
#     for o in range(len(jsp.jobs[j].operations)):
#         # find all start variables of the current operation
#         vars_of_current_operation = [x for x in all_var_names if f"b_var_j{j}o{o}" in x.name]

#         # Our equality constraint that makes sure ever operation starts exactly once!
#         mdl.add_constraint(mdl.sum(vars_of_current_operation) == 1)

# # objective funtion without penalty term 4 (was replaced by linear equality constraints above)  
# mdl.minimize(i_var_c_max + p5 * term_5 + p6 * term_6 + p7 * term_7)

In [18]:
mdl

docplex.mp.Model['jssp time indexed qubo model']

In [19]:
mdl.prettyprint()

// This file has been generated by DOcplex
// model name is: jssp time indexed qubo model
// single vars section
dvar int i_var_c_max;
dvar bool b_var_j0o0t0;
dvar bool b_var_j0o1t1;
dvar bool b_var_j1o0t0;
dvar bool b_var_j1o1t1;
dvar int i_var_slack_j0;
dvar int i_var_slack_j1;

minimize
 - 3 i_var_c_max - 2 b_var_j0o0t0 - 2 b_var_j1o0t0 + 2 i_var_slack_j0
 + 2 i_var_slack_j1 [ 2 i_var_c_max^2 - 2 i_var_c_max*b_var_j0o1t1
 - 2 i_var_c_max*b_var_j1o1t1 - 2 i_var_c_max*i_var_slack_j0
 - 2 i_var_c_max*i_var_slack_j1 + b_var_j0o0t0^2 + 2 b_var_j0o1t1^2
 + 2 b_var_j0o1t1*i_var_slack_j0 + b_var_j1o0t0^2 + 2 b_var_j1o1t1^2
 + 2 b_var_j1o1t1*i_var_slack_j1 + i_var_slack_j0^2 + i_var_slack_j1^2 ] + 6;
 
subject to {

}


-----
## Convert the mathematical model into a QUBO

In [20]:
### CREATE QUBO 

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import LinearEqualityToPenalty, IntegerToBinary

# Create the QUBO from our docplex-model ...
qp = from_docplex_mp(mdl) # create a qiskit_optimization.QuadraticProgramm from this docplex.mp.model

# Convert QP into QUBO. Only binary variables and no constraints allowed.
lin_eq_converter = LinearEqualityToPenalty(p4)
int_to_bin_converter = IntegerToBinary()

# Turn linear equality constraints into penalty terms. only necessary if we replaced a penalty term with linear equality constraints.
qp = lin_eq_converter.convert(qp) 

# Turn integers into binary vars.
qubo = int_to_bin_converter.convert(qp) 


In [21]:
qubo.prettyprint()

'Problem name: jssp time indexed qubo model\n\nMinimize\n  b_var_j0o0t0^2 + 2*b_var_j0o1t1^2 + 2*b_var_j0o1t1*i_var_slack_j0@0\n  + b_var_j1o0t0^2 + 2*b_var_j1o1t1^2 + 2*b_var_j1o1t1*i_var_slack_j1@0\n  - 2*i_var_c_max@0*b_var_j0o1t1 - 2*i_var_c_max@0*b_var_j1o1t1\n  + 2*i_var_c_max@0^2 + 4*i_var_c_max@0*i_var_c_max@1\n  - 2*i_var_c_max@0*i_var_slack_j0@0 - 2*i_var_c_max@0*i_var_slack_j1@0\n  - 2*i_var_c_max@1*b_var_j0o1t1 - 2*i_var_c_max@1*b_var_j1o1t1\n  + 2*i_var_c_max@1^2 - 2*i_var_c_max@1*i_var_slack_j0@0\n  - 2*i_var_c_max@1*i_var_slack_j1@0 + i_var_slack_j0@0^2 + i_var_slack_j1@0^2\n  - 2*b_var_j0o0t0 - 2*b_var_j1o0t0 - 3*i_var_c_max@0 - 3*i_var_c_max@1\n  + 2*i_var_slack_j0@0 + 2*i_var_slack_j1@0 + 6\n\nSubject to\n  No constraints\n\n  Binary variables (8)\n    i_var_c_max@0 i_var_c_max@1 b_var_j0o0t0 b_var_j0o1t1 b_var_j1o0t0\n    b_var_j1o1t1 i_var_slack_j0@0 i_var_slack_j1@0\n'

In [22]:
qubo

<QuadraticProgram: minimize b_var_j0o0t0^2 + 2*b_var_j0o1t1^2 + 2*b_var_j0o1t1..., 8 variables, 0 constraints, 'jssp time indexed qubo model'>

-----
# Solve the QUBO using a classical CPLEX-Solver

In [23]:
### SOLVE THE QUBO USING CPLEX

from qiskit_optimization.algorithms import CplexOptimizer

cplex_optimizer = CplexOptimizer()

cplex_result = cplex_optimizer.solve(qubo)

In [24]:
print(f"CPLEX solution and function value:")
print(f"minimum point = {cplex_result.x}")
print(f"minimum value fval = {cplex_result.fval}")

CPLEX solution and function value:
minimum point = [1. 1. 1. 1. 1. 1. 0. 0.]
minimum value fval = 2.0


In [25]:
print("CPLEX solution with variable names:")
print(f"{cplex_result.variables_dict}")

CPLEX solution with variable names:
{'i_var_c_max@0': 1.0, 'i_var_c_max@1': 1.0, 'b_var_j0o0t0': 1.0, 'b_var_j0o1t1': 1.0, 'b_var_j1o0t0': 1.0, 'b_var_j1o1t1': 1.0, 'i_var_slack_j0@0': 0.0, 'i_var_slack_j1@0': 0.0}


In [26]:
cplex_solution_schedule, cplex_makespan = create_solution_schedule_from_binary_vars(jsp, cplex_result.variables_dict)

In [27]:
print("CPLEX solution schedule")
print(cplex_solution_schedule)

CPLEX solution schedule
[[[0], [1]], [[0], [1]]]


In [28]:
print("CPLEX makespan of the solution schedule")
print(cplex_makespan)


CPLEX makespan of the solution schedule
2


In [29]:
draw_solution(jsp, solution=cplex_solution_schedule, x_max=cplex_makespan+1, title='Solution using CPlexSolver()')

### ✔️ Success! CPLEX solved this instance properly!

-----
# Solve the QUBO using QAOA!

In [30]:
from qiskit import IBMQ
from qiskit import Aer

API_TOKEN_DE = '' # <<< Enter your API_TOKEN here
API_URL_DE = 'https://auth.de.quantum-computing.ibm.com/api'


if API_TOKEN_DE == '' or API_URL_DE == '':
    solver_backend = Aer.get_backend('qasm_simulator')
    print('No API_TOKEN provided. Using a local qasm_simulator on your machine.')
else:
    IBMQ.enable_account(API_TOKEN_DE, API_URL_DE)
    provider = IBMQ.get_provider(hub='fraunhofer-de', group='fhg-all', project='ticket')
    solver_backend = provider.get_backend('ibmq_ehningen')

    

No API_TOKEN provided. Using a local qasm_simulator on your machine.


In [31]:
backend = solver_backend
backend

QasmSimulator('qasm_simulator')

In [32]:
# QAOA parameters
number_of_shots = 100000
qaoa_reps = 1

# Betas and Gammas as described in the original QAOA paper
betas =     [0] 
gammas =    [0]

In [33]:
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver

algorithm_globals.random_seed = 10598

from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)

quantum_instance = QuantumInstance(
    backend,
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
    shots=number_of_shots
)

qaoa_mes = QAOA(quantum_instance=quantum_instance, reps=qaoa_reps, initial_point=[*betas, *gammas])

In [34]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

In [35]:
qaoa_result = qaoa.solve(qubo)

In [36]:
print(f"QAOA solution and function value:")
print(f"minimum point = {qaoa_result.x}")
print(f"minimum value fval = {qaoa_result.fval}")

QAOA solution and function value:
minimum point = [1. 1. 1. 1. 1. 1. 0. 0.]
minimum value fval = 2.0


In [37]:
print("QAOA solution with variable names:")
print(f"{qaoa_result.variables_dict}")

QAOA solution with variable names:
{'i_var_c_max@0': 1.0, 'i_var_c_max@1': 1.0, 'b_var_j0o0t0': 1.0, 'b_var_j0o1t1': 1.0, 'b_var_j1o0t0': 1.0, 'b_var_j1o1t1': 1.0, 'i_var_slack_j0@0': 0.0, 'i_var_slack_j1@0': 0.0}


In [38]:
qaoa_solution_schedule, qaoa_makespan = create_solution_schedule_from_binary_vars(jsp, qaoa_result.variables_dict)


In [39]:
print("QAOA solution schedule")
print(qaoa_solution_schedule)

QAOA solution schedule
[[[0], [1]], [[0], [1]]]


In [40]:
print("QAOA makespan of the solution schedule")
print(qaoa_makespan)


QAOA makespan of the solution schedule
2


In [41]:
draw_solution(jsp, solution=qaoa_solution_schedule, x_max=qaoa_makespan+1, title='Solution using QAOA()')

### ✔️ Success! QAOA solved this instance properly!

-----
# Let's see how the QAOA-Ansatz shot samples our circuit  

In [42]:
from qiskit.providers.aer import AerSimulator
from qiskit.utils import QuantumInstance
from qiskit.opflow import CircuitSampler

quantum_instance = backend
circuit_sampler = CircuitSampler(quantum_instance)

In [43]:
from ast import And
from qiskit.circuit.library.n_local import QAOAAnsatz
from qiskit.opflow import CircuitStateFn, OperatorStateFn, VectorStateFn, DictStateFn, CircuitSampler

import plotly.graph_objects as go
import pandas as pd


def generate_bitstrings(number_bits: int):
    return [bin(k)[2:].zfill(number_bits) for k in range(2**number_bits)]


def generate_bitarrays(number_bits: int):
    return [np.array(list(bin(k)[2:].zfill(number_bits))[::-1], dtype=int) for k in range(2**number_bits)]
    
number_binary_variables = qubo.get_num_binary_vars()

ising, ising_offset = qubo.to_ising()
simulator_with_shot_noise = False

#%%
qaoa_ansatz = QAOAAnsatz(cost_operator=ising, reps=qaoa_reps)
wave_function = CircuitStateFn(qaoa_ansatz)
observable = OperatorStateFn(ising, is_measurement=True)
expectation_operator = observable @ wave_function

def run_qaoa(parameter_values):
    "parameter_values is expected to be of the form [beta_0, beta_1, ..., gamma_0, gamma_1, ...]"
    parameter_bindings = dict(zip(qaoa_ansatz.parameters, parameter_values))
    wave_function_sampled = circuit_sampler.convert(wave_function, params=parameter_bindings)
    return wave_function_sampled

#%%
# Be careful: the data type of wave_function_sampled depends if shot noise is present
wave_function_sampled = run_qaoa([*betas, *gammas])


-----
### 📊 Prepare the shot sample results for a plot

In [44]:
samples_dict = wave_function_sampled.__dict__['_primitive']
samples_dict_df = pd.DataFrame([(x, samples_dict[x]) for x in samples_dict])
samples_dict_df = samples_dict_df.rename(columns={0: "bitstring", 1: "probability"})
samples_dict_df = samples_dict_df.sort_values(by='probability', ascending=False).reset_index(drop=True)
samples_dict_df

,bitstring,probability
0,10100101,0.098821
1,11110000,0.093750
2,00000010,0.093750
3,11001110,0.093750
4,11101100,0.093750
...,...,...
248,11000011,0.031250
249,01100110,0.031250
250,01001101,0.031250
251,00111011,0.031250


In [45]:
optimal_value = cplex_result.fval

In [46]:
df_further_info = pd.DataFrame(index=generate_bitstrings(number_binary_variables))

df_further_info["bitstring"] = df_further_info.index
df_further_info["bitarray"] = generate_bitarrays(number_binary_variables)
df_further_info["cost"] = df_further_info["bitarray"].apply(qubo.objective.evaluate)
df_further_info["is_optimal"] = (df_further_info["cost"] - optimal_value).apply(lambda x: np.isclose(x, 0)) 
df_further_info = df_further_info.reset_index(drop=True)

In [47]:
df_further_info

,bitstring,bitarray,cost,is_optimal
0,00000000,"[0, 0, 0, 0, 0, 0, 0, 0]",6.0,False
1,00000001,"[1, 0, 0, 0, 0, 0, 0, 0]",5.0,False
2,00000010,"[0, 1, 0, 0, 0, 0, 0, 0]",5.0,False
3,00000011,"[1, 1, 0, 0, 0, 0, 0, 0]",8.0,False
4,00000100,"[0, 0, 1, 0, 0, 0, 0, 0]",5.0,False
...,...,...,...,...
251,11111011,"[1, 1, 0, 1, 1, 1, 1, 1]",5.0,False
252,11111100,"[0, 0, 1, 1, 1, 1, 1, 1]",18.0,False
253,11111101,"[1, 0, 1, 1, 1, 1, 1, 1]",9.0,False
254,11111110,"[0, 1, 1, 1, 1, 1, 1, 1]",9.0,False


In [48]:
# Let's join the bitstring information with the sampled probabilities. 
df_further_info_with_probabilities = pd.merge(df_further_info, samples_dict_df,  how='left', left_on='bitstring', right_on='bitstring')
df_further_info_with_probabilities['probability'].fillna(0, inplace=True) # after the left join we replace probabilities of NaNs with 0  
print(f"{df_further_info_with_probabilities}")

    bitstring                  bitarray  cost  is_optimal  probability
0    00000000  [0, 0, 0, 0, 0, 0, 0, 0]   6.0       False     0.044194
1    00000001  [1, 0, 0, 0, 0, 0, 0, 0]   5.0       False     0.031250
2    00000010  [0, 1, 0, 0, 0, 0, 0, 0]   5.0       False     0.093750
3    00000011  [1, 1, 0, 0, 0, 0, 0, 0]   8.0       False     0.076547
4    00000100  [0, 0, 1, 0, 0, 0, 0, 0]   5.0       False     0.082680
..        ...                       ...   ...         ...          ...
251  11111011  [1, 1, 0, 1, 1, 1, 1, 1]   5.0       False     0.093750
252  11111100  [0, 0, 1, 1, 1, 1, 1, 1]  18.0       False     0.069877
253  11111101  [1, 0, 1, 1, 1, 1, 1, 1]   9.0       False     0.044194
254  11111110  [0, 1, 1, 1, 1, 1, 1, 1]   9.0       False     0.031250
255  11111111  [1, 1, 1, 1, 1, 1, 1, 1]   4.0       False     0.082680

[256 rows x 5 columns]


In [49]:
df_further_info_with_probabilities_sorted = df_further_info_with_probabilities.sort_values(by='probability', ascending=False)
df_further_info_with_probabilities_sorted[:10]

,bitstring,bitarray,cost,is_optimal,probability
165,10100101,"[1, 0, 1, 0, 0, 1, 0, 1]",7.0,False,0.098821
2,00000010,"[0, 1, 0, 0, 0, 0, 0, 0]",5.0,False,0.093750
251,11111011,"[1, 1, 0, 1, 1, 1, 1, 1]",5.0,False,0.093750
240,11110000,"[0, 0, 0, 0, 1, 1, 1, 1]",15.0,False,0.093750
236,11101100,"[0, 0, 1, 1, 0, 1, 1, 1]",19.0,False,0.093750
206,11001110,"[0, 1, 1, 1, 0, 0, 1, 1]",8.0,False,0.093750
155,10011011,"[1, 1, 0, 1, 1, 0, 0, 1]",4.0,False,0.088388
49,00110001,"[1, 0, 0, 0, 1, 1, 0, 0]",4.0,False,0.088388
54,00110110,"[0, 1, 1, 0, 1, 1, 0, 0]",3.0,False,0.088388
122,01111010,"[0, 1, 0, 1, 1, 1, 1, 0]",7.0,False,0.088388


-----
### 📉 Plot the sampled results and optimal solutions

In [50]:
df_to_be_ploted = df_further_info_with_probabilities_sorted

#%%
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_to_be_ploted['bitstring'],
    y=df_to_be_ploted["probability"],
    line_color="black",
    line_width=1.0,
    name=f"probability of a given bitstring"
))
fig.add_trace(go.Scatter(
    x=df_to_be_ploted[df_to_be_ploted["is_optimal"]]['bitstring'],
    y=df_to_be_ploted[df_to_be_ploted["is_optimal"]]["probability"],
    mode="markers",
    marker_symbol="arrow-down",
    marker_size=12,
    marker_color="red",
    name="optimal solution"
))
fig.update_xaxes(
    tickfont_size=1,
    title='bitstring'
)
fig.update_yaxes(
    title="probability",
    title_font_size=12,
    
)
title_betas = "".join([f"{beta:.3f}, " for beta in betas])
title_gammas = "".join([f"{gamma:.3f}, " for gamma in gammas])
fig.update_layout(
    title=f"JSP-Instance: {jsp_filepath}, \t Backend: {backend}<br>" \
    f"" \
    f"QAOA: p={qaoa_reps}, shots: {number_of_shots}<br>" \
    f"penalty factors: p4={p4}, p5={p5}, p6={p6}, p7={p7} <br>" \
        + "betas: " + title_betas + "gammas: " + title_gammas,
    title_font_size=12
)

---

(c) Author: Arthur Grigorjan, arthur.grigrojan@ipa.fraunhofer.de

---

**Copyright 2023 Arthur Grigorjan, Fraunhofer IPA**

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.